In [158]:
#http://www.tensorfly.cn/tfdoc/tutorials/mnist_beginners.html
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1f2bad048>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x12b730710>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1cd7d0588>)

In [159]:
#设置变量
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])
#dropout比例
keep_prob=tf.placeholder(tf.float32)
#增加学习率变量，训练过程中改变它
LEARN_RATE=1e-3
learn_rate=tf.Variable(LEARN_RATE,dtype=tf.float32)

Weights_L1=tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
biases_L1=tf.Variable(tf.zeros([500])+0.1)
L1=tf.nn.tanh(tf.matmul(x,Weights_L1)+biases_L1)
L1_drop=tf.nn.dropout(L1,keep_prob)

Weights_L2=tf.Variable(tf.truncated_normal([500,500],stddev=0.1))
biases_L2=tf.Variable(tf.zeros([500])+0.1)
L2=tf.nn.tanh(tf.matmul(L1_drop,Weights_L2)+biases_L2)
L2_drop=tf.nn.dropout(L2,keep_prob)

Weights_L3=tf.Variable(tf.truncated_normal([500,200],stddev=0.1))
biases_L3=tf.Variable(tf.zeros([200])+0.1)
L3=tf.nn.tanh(tf.matmul(L2_drop,Weights_L3)+biases_L3)
L3_drop=tf.nn.dropout(L3,keep_prob)

Weights_L4=tf.Variable(tf.truncated_normal([200,10],stddev=0.1))
biases_L4=tf.Variable(tf.zeros([10])+0.1)

In [160]:
#构造模型
prediction=tf.nn.softmax(tf.matmul(L3_drop,Weights_L4)+biases_L4)

#loss
# loss=tf.square(prediction-y)
# loss=tf.reduce_mean(tf.square(prediction-y))
loss=tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction)
# loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))

train_step=tf.train.AdamOptimizer(learning_rate=learn_rate).minimize(loss)

In [161]:
#评估
#correct_prediction是一个bool list
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#准确率
"""
tf.cast:转化类型 True->1.0;False->0.0
tf.reduce_mean:求平均值，即准确率
"""
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [162]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [163]:
#训练
drop_ratio=1
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys,keep_prob:drop_ratio})
    if i%200==0:
        sess.run(tf.assign(learn_rate,LEARN_RATE*(0.95**(i/200))))
        test_accuracy=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        train_accuracy=sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print('test_data:'+str(test_accuracy)+",train_data:"+str(train_accuracy))

test_data:0.2355,train_data:0.234655
test_data:0.9319,train_data:0.932564
test_data:0.951,train_data:0.952782
test_data:0.9583,train_data:0.960436
test_data:0.9598,train_data:0.965636
test_data:0.9631,train_data:0.968782
test_data:0.9637,train_data:0.971491
test_data:0.9636,train_data:0.970982
test_data:0.9635,train_data:0.9722
test_data:0.9697,train_data:0.9778
test_data:0.9688,train_data:0.979545
test_data:0.9673,train_data:0.978673
test_data:0.9724,train_data:0.982382
test_data:0.9736,train_data:0.9816
test_data:0.9728,train_data:0.9828
